In [ ]:
from utils import *
from cluster import *
import gensim
from gensim.models import KeyedVectors
import codecs
import numpy as np
from numpy import linalg as LA
import scipy
import os, json
import operator
import pickle

In [ ]:
Loading word2vec embeddings
w2v, w2v_w2i, w2v_vocab = load_w2v("data/GoogleNews-vectors-negative300.bin")

***Plain word2vec WEAT***

In [ ]:
# WEAT embeddings
# load word lists for tests
# start with the male and female names
A = [name.lower() for name in WEAT_words['A']]
B = [name.lower() for name in WEAT_words['B']]
# career and family
C = WEAT_words['C']
D = WEAT_words['D']
# math and arts
E = WEAT_words['E']
F = WEAT_words['F']
# science and arts
G = WEAT_words['G']
H = WEAT_words['H']

In [ ]:
# calculate effect size and p value for career and family plain w2v
print("PLAIN W2V")
print('Career and Family')
print("Effect size:",effect_size(A, B, C, D, w2v, w2v_w2i, w2v_vocab))
print("P-value:",p_value_test(A, B, C, D, w2v, w2v_w2i, w2v_vocab))

# calculate effect size and p value for math and arts plain w2v
print('Math and Arts')
print("Effect size:",effect_size(A, B, E, F, w2v, w2v_w2i, w2v_vocab))
print("P-value:",p_value_test(A, B, E, F, w2v, w2v_w2i, w2v_vocab))

# calculate effect size and p value for science and arts plain w2v
print('Science and Arts')
print("Effect size:",effect_size(A, B, G, H, w2v, w2v_w2i, w2v_vocab))
print("P-value:",p_value_test(A, B, G, H, w2v, w2v_w2i, w2v_vocab))

***Clustering for plain Word2Vec***

In [ ]:
# limit vocab by excluding words that 'should' have gender bias
gender_specific = []

with open("data/male_words.txt") as f:
    for l in f:
        gender_specific.append(l.strip())
with open("data/female_words.txt") as f:
    for l in f:
        gender_specific.append(l.strip())

with codecs.open("data/gender_specific_full.json") as f:
    gender_specific.extend(json.load(f))

w2v = decentralize(w2v)
w2v = normalize(w2v)
w2v_vocab_limit, w2v_wv_limit, w2v_w2i_limit = limit_vocab(w2v, w2v_w2i, w2v_vocab, exclude=gender_specific)
# dbl_glove_vocab_limit, dbl_glove_wv_limit, dbl_glove_w2i_limit = limit_vocab(dbl_glove_wv, dbl_glove_w2i, dbl_glove_vocab, exclude=gender_specific)

In [ ]:
# get most biased words
he_vector = w2v[w2v_w2i['he'], :]
she_vector = w2v[w2v_w2i['she'], :]
biased_words = compute_word_bias(w2v_wv_limit, w2v_w2i_limit, w2v_vocab_limit, he_vector, she_vector)

In [ ]:
# cluster using limited vocabulary
for n in [100, 500, 1000]:
  print("\n Precision for plain word2vec for",n,"words")
  my_cluster(w2v_wv_limit, w2v_w2i_limit, 1, w2v_vocab_limit, biased_words, num_biased_words=n)

# LOAD HARD-DEBIAS

In [ ]:
from hard_debias import *

In [ ]:
hard_w2v = hard_debias_w2v(w2v, w2v_w2i, w2v_vocab,"data/definitional_pairs.json")
print("Hard-debiased vectors loaded")

WEAT FOR HARD-DEBIAS

In [ ]:
# calculate effect size and p value for career and family hard w2v
print("HARD-DEBIASED W2V")
print('Career and Family')
print("Effect-size:",effect_size(A, B, C, D, hard_w2v, w2v_w2i, w2v_vocab))
print("P-value:",p_value_test(A, B, C, D, hard_w2v, w2v_w2i, w2v_vocab))

# calculate effect size and p value for math and arts hard w2v
print('Math and Arts')
print("Effect-size:",effect_size(A, B, E, F, hard_w2v, w2v_w2i, w2v_vocab))
print("P-value:",p_value_test(A, B, E, F, hard_w2v, w2v_w2i, w2v_vocab))

# calculate effect size and p value for science and arts hard w2v
print('Science and Arts')
print("Effect-size:",effect_size(A, B, G, H, hard_w2v, w2v_w2i, w2v_vocab))
print("P-value:",p_value_test(A, B, G, H, hard_w2v, w2v_w2i, w2v_vocab))

***Clustering for hard debiased Word2Vec***

In [ ]:
# limit vocab by excluding words that 'should' have gender bias
gender_specific = []

with open("data/male_words.txt") as f:
    for l in f:
        gender_specific.append(l.strip())
with open("data/female_words.txt") as f:
    for l in f:
        gender_specific.append(l.strip())

with codecs.open("data/gender_specific_full.json") as f:
    gender_specific.extend(json.load(f))

hard_w2v = decentralize(hard_w2v)
hard_w2v = normalize(hard_w2v)
hard_w2v_vocab_limit, hard_w2v_wv_limit, hard_w2v_w2i_limit = limit_vocab(hard_w2v, w2v_w2i, w2v_vocab, exclude=gender_specific)

In [ ]:
# get most biased words
he_vector = hard_w2v[w2v_w2i['he'], :]
she_vector = hard_w2v[w2v_w2i['she'], :]
biased_words = compute_word_bias(hard_w2v_wv_limit, hard_w2v_w2i_limit, hard_w2v_vocab_limit, he_vector, she_vector)

In [ ]:
# cluster using limited vocabulary
for n in [100, 500, 1000]:
  print("\n Precision for hard-debiased word2vec for",n,"words")
  my_cluster(hard_w2v_wv_limit, hard_w2v_w2i_limit, 1, hard_w2v_vocab_limit, biased_words, num_biased_words=n)